In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pandas.core.common import SettingWithCopyWarning
import numpy as np
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

### Read data and clean data

In [3]:
individual_profile = pd.read_csv("./data/University of Michigan-Ann Arbor.csv")

## set school name 
school_name = "University of Michigan-Ann Arbor"

In [5]:
## drop duplicates 
individual_profile = individual_profile.drop_duplicates(subset=['current_location','latest_Education','current_work','previous_work'], keep='last')
individual_profile.head()

,Unnamed: 0,current_location,latest_Education,current_work,previous_work,majro,search_school
1,1,Washington DC-Baltimore Area,University of Michigan - Ann Arbor,Resident at Urban Teachers,Religious School Teacher at Beth Israel Congre...,NaN,University of Michigan-Ann Arbor
2,2,"Swartz Creek, Michigan, United States",Baker College,Neurophysiologist at SpecialtyCare,Neurodiagnostic Technologist at MidMichigan He...,Movement Science,University of Michigan-Ann Arbor
3,3,"Ojai, California, United States",Northwestern University,Senior Analyst at Decision Resources Group,Market Analyst at Decision Resources Group,Journalism,University of Michigan-Ann Arbor
4,4,"Washington, District of Columbia, United States",University of Michigan - Ann Arbor,Lead Content Creator at Health Law Institute,Policy Intern at US Global Leadership Coalition,NaN,University of Michigan-Ann Arbor
5,5,"Ann Arbor, Michigan, United States",University of Michigan - Ann Arbor,NaN,Researcher at Industrial Technology Research I...,Nuclear Engineering and Radiological Sciences,University of Michigan-Ann Arbor
...,...,...,...,...,...,...,...
881,881,Buffalo-Niagara Falls Area,University of Michigan,Division Chief Kidney and Pancreas Transplanta...,NaN,Clinical Outcomes and Statistics,University of Michigan-Ann Arbor
882,882,Detroit Metropolitan Area,University of Michigan - Ann Arbor,Google Associate Partner Brand Manager via Hug...,Cultural Brand Strategist at Fiverr Freelancer...,Sport and Fitness Administration/Management,University of Michigan-Ann Arbor
883,883,Detroit Metropolitan Area,University of Michigan - Ann Arbor,AME Director - US & Canada at Lear Corporation,Engineering Group Manager AME US & Canada at L...,Industrial Engineering,University of Michigan-Ann Arbor
884,884,"Vadodara, Gujarat, India",Rutgers University,Researcher PHD Student at Rutgers University,Advanced Manufacturing Engineer- Industry 4.0 ...,Mechanical Engineering,University of Michigan-Ann Arbor


## Matching industry
### 1. Get current and previous company name

In [6]:
individual_profile.loc[:,'current_company'] = individual_profile['current_work'].str.split('at ').str[-1]
individual_profile.loc[:,'previous_company'] = individual_profile['previous_work'].str.split('at ').str[-1]

In [7]:
individual_profile = individual_profile.drop(columns =['Unnamed: 0'])## drop column 
individual_profile.loc[:,'current_company_industry'] = np.nan
pd.set_option('display.max_rows', 500)
individual_profile

,index,current_location,latest_Education,current_work,previous_work,majro,search_school,current_company,previous_company,current_company_industry
1,1,Washington DC-Baltimore Area,University of Michigan - Ann Arbor,Resident at Urban Teachers,Religious School Teacher at Beth Israel Congre...,NaN,University of Michigan-Ann Arbor,Urban Teachers,Beth Israel Congregation - Ann Arbor,NaN
2,2,"Swartz Creek, Michigan, United States",Baker College,Neurophysiologist at SpecialtyCare,Neurodiagnostic Technologist at MidMichigan He...,Movement Science,University of Michigan-Ann Arbor,SpecialtyCare,MidMichigan Health,NaN
3,3,"Ojai, California, United States",Northwestern University,Senior Analyst at Decision Resources Group,Market Analyst at Decision Resources Group,Journalism,University of Michigan-Ann Arbor,Decision Resources Group,Decision Resources Group,NaN
4,4,"Washington, District of Columbia, United States",University of Michigan - Ann Arbor,Lead Content Creator at Health Law Institute,Policy Intern at US Global Leadership Coalition,NaN,University of Michigan-Ann Arbor,Health Law Institute,US Global Leadership Coalition,NaN
5,5,"Ann Arbor, Michigan, United States",University of Michigan - Ann Arbor,NaN,Researcher at Industrial Technology Research I...,Nuclear Engineering and Radiological Sciences,University of Michigan-Ann Arbor,NaN,Industrial Technology Research Institute (工業技術...,NaN
...,...,...,...,...,...,...,...,...,...,...
881,881,Buffalo-Niagara Falls Area,University of Michigan,Division Chief Kidney and Pancreas Transplanta...,NaN,Clinical Outcomes and Statistics,University of Michigan-Ann Arbor,Erie County Medical Center Corporation,NaN,NaN
882,882,Detroit Metropolitan Area,University of Michigan - Ann Arbor,Google Associate Partner Brand Manager via Hug...,Cultural Brand Strategist at Fiverr Freelancer...,Sport and Fitness Administration/Management,University of Michigan-Ann Arbor,Huge,Fiverr Freelancers (Buyer and Seller),NaN
883,883,Detroit Metropolitan Area,University of Michigan - Ann Arbor,AME Director - US & Canada at Lear Corporation,Engineering Group Manager AME US & Canada at L...,Industrial Engineering,University of Michigan-Ann Arbor,Lear Corporation,Lear Corporation,NaN
884,884,"Vadodara, Gujarat, India",Rutgers University,Researcher PHD Student at Rutgers University,Advanced Manufacturing Engineer- Industry 4.0 ...,Mechanical Engineering,University of Michigan-Ann Arbor,Rutgers University,STANLEY Engineered Fastening,NaN


In [8]:
## open browser 
browser = webdriver.Chrome('your local path/chromedriver')
browser.get('https://web-p-ebscohost-com.proxy.uchicago.edu/ehost/search/advanced?vid=13&sid=a64a2a8c-e647-4836-8d08-8ad530f8d210%40redis')

<ipython-input-8-a48ea40b4e9f>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('/Users/jarvis/Desktop/winter 22/CS for SS2/fianl project/final-project-ocean-s-4/chromedriver')


### Scrapping industry of current company 

In [ ]:
## input search term 
for index, com in individual_profile[1510:].iterrows():
    index, search_term = index,com['current_company']
    if not pd.isnull(search_term):
        print(index) ## to keep track of interruption
        browser.find_element(By.ID,"SearchTerm1").clear()
        search_box = browser.find_element(By.ID,"SearchTerm1")
        search_box.send_keys(search_term)
        search_box.send_keys(Keys.RETURN)
        ## click on result 
        try:
            table = browser.find_element(By.XPATH,"//*[@id='companyResults']/tbody")
            rows = table.find_element(By.XPATH,"//*[@id='companyResults']/tbody/tr[2]")
            rows.find_element(By.XPATH,'//*[@id="companyResults"]/tbody/tr[2]/td[2]/a').click()
            src = browser.page_source
            soup = BeautifulSoup(src, 'lxml')
            industry_info = soup.find_all('dl', {'class': 'data-details definition-list'})
            try:
                for ic in industry_info[2].find_all('a',{'data-auto':'link'}):
                    individual_profile.loc[index,'current_company_industry'] = ic.text
            except IndexError:
                pass
        except NoSuchElementException:
            pass


1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1550
1551
1552
1553
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572


In [65]:
pd.set_option('display.max_rows', 500)
# individual_profile.head(15)
individual_profile[1300:1500]

,current_location,latest_Education,current_work,previous_work,majro,search_school,current_company,previous_company,current_company_industry,previous_company_industry
1300,"Porto, Porto, Portugal",University of Toronto,Translator at RWS Group,"Media Manager, Consumer GSK, Brazil at GSK",Translation and Language Services,University of Michigan-Ann Arbor,RWS Group,GSK,"541512 Systems integration design services, co...",NaN
1301,"New York, New York, United States",University of Michigan - Ann Arbor,NaN,89.1 WBOI NPR News Intern at Northern Indiana ...,"History, Philosophy",University of Michigan-Ann Arbor,NaN,Northern Indiana Public Radio,NaN,NaN
1302,"Philadelphia, Pennsylvania, United States",The Peabody Institute of The Johns Hopkins Uni...,General Manager at Convergence Ensemble,Director of Upper School Choirs at Charlotte L...,Vocal Performance,University of Michigan-Ann Arbor,Convergence Ensemble,Charlotte Latin School,NaN,NaN
1303,New York City Metropolitan Area,University of Michigan - Ann Arbor,NaN,Capital Markets Development Intern at Trammell...,NaN,University of Michigan-Ann Arbor,NaN,Trammell Crow Residential,NaN,NaN
1304,Peru,Facultad Latinoamericana de Ciencias Sociales,Member at Foro Educativo,Gerente de Gestion de Calidad en ONPE at Ofici...,Social Sciences,University of Michigan-Ann Arbor,Foro Educativo,Oficina Nacional de Procesos Electorales (ONPE),NaN,NaN
1305,"East Lansing, Michigan, United States",University of Michigan - Ann Arbor,NaN,Data Science Intern at Delta Dental of Michigan,Computer Science and Political Science,University of Michigan-Ann Arbor,NaN,Delta Dental of Michigan,NaN,NaN
1306,United States,University of Michigan,Systems Engineer at Ford Motor Company,Autonomous Vehicle Risk Modeling Intern at Ret...,Electrical and Electronics Engineering,University of Michigan-Ann Arbor,Ford Motor Company,Retrospect,532112 Van (passenger) leasing,NaN
1307,"Houston, Texas, United States",University of Michigan - Ann Arbor,Meteorologist-retired at None,Senior Marine Meteorologist at DTN,Meteorology,University of Michigan-Ann Arbor,None,DTN,"445299 Water stores, bottled",NaN
1308,"Atlanta, Georgia, United States",University of Michigan - Ann Arbor,Associate at Rule Joy Trammell Rubio,Intern architect at The Y Studio,Architecture Design & Res MS Digital Technologies,University of Michigan-Ann Arbor,Rule Joy Trammell Rubio,The Y Studio,541310 Building architectural design services,NaN
1309,"Santa Clara, California, United States",San Jose State University,Palliative Medical Social Worker at Kaiser Per...,MSW Intern at DFCS/Silicon Valley Children's F...,Mental and Social Health Services and Allied P...,University of Michigan-Ann Arbor,Kaiser Permanente,"DFCS/Silicon Valley Children's Fund, Emerging ...",524114 Medical service plans without providing...,NaN


### Scrapping industry of previous company

In [24]:
individual_profile.loc[:,'previous_company_industry'] = np.nan

In [ ]:
## input search term 
for index, com in individual_profile.iterrows():
    index, search_term = index,com['previous_company']
    if not pd.isnull(search_term):
        browser.find_element(By.ID,"SearchTerm1").clear()
        search_box = browser.find_element(By.ID,"SearchTerm1")
        search_box.send_keys(search_term)
        search_box.send_keys(Keys.RETURN)
        ## click on result 
        try:
            table = browser.find_element(By.XPATH,"//*[@id='companyResults']/tbody")
            rows = table.find_element(By.XPATH,"//*[@id='companyResults']/tbody/tr[2]")
            rows.find_element(By.XPATH,'//*[@id="companyResults"]/tbody/tr[2]/td[2]/a').click()
            src = browser.page_source
            soup = BeautifulSoup(src, 'lxml')
            industry_info = soup.find_all('dl', {'class': 'data-details definition-list'})
            try:
                for ic in industry_info[2].find_all('a',{'data-auto':'link'}):
                    individual_profile.loc[index,'previous_company_industry'] = ic.text
            except IndexError:
                pass
        except NoSuchElementException:
            pass


In [26]:
filename = str(school_name)+"_with_industry_code.csv"
individual_profile.to_csv(filename,index = True)